In [1]:

import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch_geometric.utils import to_undirected
import torch_geometric.datasets as geom_datasets

from topomodelx.nn.hypergraph.unisage import UniSAGE
import warnings

warnings.filterwarnings("ignore")

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Train a Uni-sage TNN

# Pre-processing

## Import data ##

The first step is to import the dataset, Cora, a benchmark classification datase. We then lift the graph into our domain of choice, a hypergraph.


In [3]:
cora = geom_datasets.Planetoid(root="/TopoModelX/data/cora", name="Cora")
data = cora.data

x_0s = data.x
y = data.y
edge_index = data.edge_index

train_mask = data.train_mask
val_mask = data.val_mask
test_mask = data.test_mask

## Define neighborhood structures and lift into hypergraph domain. ##

Now we retrieve the neighborhood structure (i.e. their representative matrice) that we will use to send messges from node to hyperedges. In the case of this architecture, we need the boundary matrix (or incidence matrix) $B_1$ with shape $n_\text{nodes} \times n_\text{edges}$.

In citation Cora dataset we lift graph structure to the hypergraph domain by creating hyperedges from 1-hop graph neighbourhood of each node. 


In [4]:
# Ensure the graph is undirected (optional but often useful for one-hop neighborhoods).
edge_index = to_undirected(edge_index)

# Create a list of one-hop neighborhoods for each node.
one_hop_neighborhoods = []
for node in range(data.num_nodes):
    # Get the one-hop neighbors of the current node.
    neighbors = data.edge_index[1, data.edge_index[0] == node]

    # Append the neighbors to the list of one-hop neighborhoods.
    one_hop_neighborhoods.append(neighbors.numpy())

# Detect and eliminate duplicate hyperedges.
unique_hyperedges = set()
hyperedges = []
for neighborhood in one_hop_neighborhoods:
    # Sort the neighborhood to ensure consistent comparison.
    neighborhood = tuple(sorted(neighborhood))
    if neighborhood not in unique_hyperedges:
        hyperedges.append(list(neighborhood))
        unique_hyperedges.add(neighborhood)    

Additionally we print the statictis associated with obtained incidence matrix

In [5]:
# Calculate hyperedge statistics.
hyperedge_sizes = [len(he) for he in hyperedges]
min_size = min(hyperedge_sizes)
max_size = max(hyperedge_sizes)
mean_size = np.mean(hyperedge_sizes)
median_size = np.median(hyperedge_sizes)
std_size = np.std(hyperedge_sizes)
num_single_node_hyperedges = sum(np.array(hyperedge_sizes) == 1)

# Print the hyperedge statistics.
print(f'Hyperedge statistics: ')
print('Number of hyperedges without duplicated hyperedges', len(hyperedges))
print(f'min = {min_size}, ')
print(f'max = {max_size}, ')
print(f'mean = {mean_size}, ')
print(f'median = {median_size}, ')
print(f'std = {std_size}, ')
print(f'Number of hyperedges with size equal to one = {num_single_node_hyperedges}')


Hyperedge statistics: 
Number of hyperedges without duplicated hyperedges 2581
min = 1, 
max = 168, 
mean = 4.003099573808601, 
median = 3.0, 
std = 5.327622607829558, 
Number of hyperedges with size equal to one = 412


Construct incidence matrix

In [6]:
max_edges = len(hyperedges)
incidence_1 = np.zeros((x_0s.shape[0], max_edges))
for col, neighibourhood in enumerate(hyperedges):
    for row in neighibourhood:
        incidence_1[row, col] = 1

assert all(incidence_1.sum(0)>0) == True, "Some hyperedges are empty"
assert all(incidence_1.sum(1)>0) == True, "Some nodes are not in any hyperedges"
incidence_1 = torch.Tensor(incidence_1).to_sparse_coo()

# Create the Neural Network

Using the UniSAGELayer class, we create a neural network with stacked layers.

In [7]:
in_channels = x_0s.shape[1]
hidden_channels = 128
out_channels = torch.unique(y).shape[0]
task_level = "graph" if out_channels==1 else "node"
n_layers=1
model = UniSAGE(
    in_channels=in_channels,
    hidden_channels=hidden_channels,
    out_channels=out_channels, 
    input_drop=0.2,
    layer_drop=0.2,
    n_layers=n_layers,
    task_level=task_level,
).to(device)

# Train the Neural Network

We specify the model, the loss, and an optimizer.

In [8]:
# Optimizer and loss
opt = torch.optim.Adam(model.parameters(), lr=0.01)

# Categorial cross-entropy loss
loss_fn = torch.nn.CrossEntropyLoss()

# Accuracy
acc_fn = lambda y, y_hat: (y == y_hat).float().mean()

In [9]:
x_0s = torch.tensor(x_0s)
x_0s, incidence_1, y = (
            x_0s.float().to(device),
            incidence_1.float().to(device),
            torch.tensor(y, dtype=torch.long).to(device),
        )

The following cell performs the training, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

In [10]:
torch.manual_seed(0)
test_interval = 5
num_epochs = 30

epoch_loss = []
for epoch_i in range(1, num_epochs + 1):
    
    model.train()

    opt.zero_grad()
    
    # Extract edge_index from sparse incidence matrix
    y_hat = model(x_0s, incidence_1)
    loss = loss_fn(y_hat[train_mask], y[train_mask])

    loss.backward()
    opt.step()
    epoch_loss.append(loss.item())

    

    if epoch_i % test_interval == 0:
        
        model.eval()
        y_hat = model(x_0s, incidence_1)

        loss = loss_fn(y_hat[train_mask], y[train_mask])
        print(f"Epoch: {epoch_i} ")
        print(f"Train_loss: {np.mean(epoch_loss):.4f}, acc: {acc_fn(y_hat[train_mask].argmax(1), y[train_mask]):.4f}",flush=True)

        loss = loss_fn(y_hat[val_mask], y[val_mask])
        
        print(f"Val_loss: {loss:.4f}, Val_acc: {acc_fn(y_hat[val_mask].argmax(1), y[val_mask]):.4f}", flush=True)

        loss = loss_fn(y_hat[test_mask], y[test_mask])
        print(f"Test_loss: {loss:.4f}, Test_acc: {acc_fn(y_hat[test_mask].argmax(1), y[test_mask]):.4f}", flush=True)

Epoch: 5 
Train_loss: 1.1164, acc: 0.9714
Val_loss: 0.9324, Val_acc: 0.7820
Test_loss: 1.0056, Test_acc: 0.7930
Epoch: 10 
Train_loss: 0.6460, acc: 0.9929
Val_loss: 1.4363, Val_acc: 0.7640
Test_loss: 1.7823, Test_acc: 0.7450
Epoch: 15 
Train_loss: 0.4511, acc: 1.0000
Val_loss: 1.6066, Val_acc: 0.7520
Test_loss: 1.7711, Test_acc: 0.7620
Epoch: 20 
Train_loss: 0.3402, acc: 1.0000
Val_loss: 2.0556, Val_acc: 0.7520
Test_loss: 2.4377, Test_acc: 0.7490


Epoch: 25 
Train_loss: 0.2747, acc: 1.0000
Val_loss: 2.1278, Val_acc: 0.7460
Test_loss: 2.4358, Test_acc: 0.7490
Epoch: 30 
Train_loss: 0.2295, acc: 1.0000
Val_loss: 2.2190, Val_acc: 0.7500
Test_loss: 2.5519, Test_acc: 0.7460
